# Retroactive changes
* Notebooks from now on will be on their own folders

This makes is so much easier to keep track of dependencies, and we also resolve the issue of IfCntk overriding the local <code>main.group.fsx</code> path.

# Preparing workspace

Our new dependency for this notebook is Newtonsoft.Json, since json is so easy to use from javascript.

In [ ]:
#load "Paket.fsx"
 
Paket.Dependencies.Install """
framework: netstandard2.0
generate_load_scripts: true
storage: none
source https://nuget.org/api/v2
nuget CNTK.CPUOnly
nuget Newtonsoft.Json
nuget MathNet.Numerics
nuget MathNet.Numerics.FSharp
"""

In [1]:
#r "netstandard"
#r @"..\bin\Cntk.Core.Managed-2.6.dll"
#load "../Graphviz/.paket/load/main.group.fsx"

In [2]:
open System
open System.IO

Environment.GetEnvironmentVariable("PATH")
|> fun path -> sprintf "%s%c%s" path (Path.PathSeparator) (Path.GetFullPath("bin"))
|> fun path -> Environment.SetEnvironmentVariable("PATH", path)

open CNTK
DeviceDescriptor.UseDefaultDevice().Type
|> printfn "Congratulations, you are using CNTK for: %A" 

Congratulations, you are using CNTK for: CPU


In [3]:
let device = CNTK.DeviceDescriptor.CPUDevice
let dataType = CNTK.DataType.Float
let initialization = CNTKLib.GlorotUniformInitializer(1.0)
let input_dim, num_output_classes = 2,2

#load "../fsx/CntkHelpers.fsx"
open CntkHelpers

# Building a sample model

Let's create a simple classifier with one hidden layer of 50 nodes that outputs to a softmax function:

In [4]:
let input = Variable.InputVariable(shape [|input_dim|], dataType, "Features")
let label = Variable.InputVariable(shape [|num_output_classes|], dataType, "Labels")
let z = 
    fullyConnectedClassifierNet input [50] num_output_classes CNTKLib.Sigmoid
    |> (Var>>CNTKLib.Softmax)    

# The Search for Nodes

The only way on offer for retrieving the model's consituent components seems to be retrieving <code>Function</code> objects by name, using <code>Function.FindByName/FindAllByName</code>. Maintaining a naming scheme while building a model in CNTK inmy opinion not especially expedient at the best of times, and a downright pain if you are training to implement functional idioms.

In addition, if you are trying to catch a glimpse of the internal workings of a CNTK model it's likely because you had no hand in creating it in the first place. A way to blindly extract network achitecture seems to be necessary.

Recursively accumulating connected <code>Variables</code> and checking the <code>Owner</code> property appears to be a simple way to extract a model's component <code>Functions</code>. 

In [5]:
/// Graph search for CNTK.Function objects.
/// We use the Input & Output lists in a Function
/// to find other Function objects by checking
/// the Variable's Owner property.
/// <remarks> CNTK helper function </remarks>
let decomposeFunction (root: Function) = 
    let visited = System.Collections.Generic.Dictionary<string, Function>()            

    let rec expand (f: Function) = 
        match visited.ContainsKey(f.Uid) with
        | true -> Seq.empty
        | false -> 
            visited.Add(f.Uid, f)
            seq {
                yield f
                yield! 
                    seq { yield! f.Inputs
                          yield! f.Outputs }
                    |> Seq.map (fun v -> v.Owner)      
                    |> Seq.filter (not<<isNull)
                    |> Seq.collect expand
            }        

    Array.ofSeq (expand root)

In [6]:
// Example: 
decomposeFunction z |> Array.map (fun f -> f.AsString())

[|"Composite(Softmax): Input('Features', [2], [*, #]) -> Output('Softmax21_Output_0', [2], [*, #])";
  "Softmax: Output('+', [2], [*, #]) -> Output('Softmax21_Output_0', [2], [*, #])";
  "Plus: Output('@', [2], [*, #]) -> Output('+', [2], [*, #])";
  "Times: Output('StableSigmoid10_Output_0', [50], [*, #]) -> Output('@', [2], [*, #])";
  "StableSigmoid: Output('+', [50], [*, #]) -> Output('StableSigmoid10_Output_0', [50], [*, #])";
  "Plus: Output('@', [50], [*, #]) -> Output('+', [50], [*, #])";
  "Times: Input('Features', [2], [*, #]) -> Output('@', [50], [*, #])"|]

<div class="alert alert-info"><p>This is a brute force algorithm. Perhaps as the complexity of our models increases we will have to revise, but for now it does fine.</p></div>

# Converting to dot notation

The great thing about GraphViz is that it does all the heavy lifting of actually building and arranging the graph, as long as we provide it with the pairs of connected nodes (in no particular order). Any additional styling information can be declared separately.

Thusly creating dot notation for a CNTK model is as simple as enumerating the component functions and declare edges according to the content of each CNTK.Function object's Input & Output Variable list, and then another pass to provide styling for each node according to its properties. 

For styling I mostly referred to the [original CNTK graph logging implementation](https://github.com/Microsoft/CNTK/blob/master/bindings/python/cntk/logging/graph.py).

In [16]:
/// Mappin the connections to and from 
/// a CNTK.Function object in GraphViz 
/// dot notation.
/// https://github.com/Microsoft/CNTK/blob/master/bindings/python/cntk/logging/graph.py
/// <remarks> CNTK helper function </remarks>
let extractGraphVizDotNotation (f: Function) = 
        let varText (v:Variable) = (if String.IsNullOrEmpty v.Name then v.Uid else v.Name)// + "\\n" + v.Shape.AsString()
        let funText (f: Function) = if String.IsNullOrEmpty f.Name then f.Uid else f.Name
    
        let varLabel (v: Variable) = sprintf "%s [label=\"%s\"];" v.Uid (varText v)
        let funLabel (f: Function) = sprintf "%s [label=\"%s\"];" f.Uid (funText f)

        let varShape (v: Variable) =
            match v with
            | _ when v.IsInput -> sprintf "%s [shape=invhouse, color=yellow];" v.Uid
            | _ when v.IsOutput -> sprintf "%s [shape=invhouse, color=gray];" v.Uid
            | _ when v.IsPlaceholder -> sprintf "%s [shape=invhouse, color=yellow];" v.Uid
            | _ when v.IsParameter -> sprintf "%s [shape=diamond, color=green];" v.Uid
            | _ when v.IsConstant -> sprintf "%s [shape=rectangle, color=lightblue];" v.Uid
            | _ -> sprintf "%s [shape=circle, color=purple];" v.Uid

        let funShape (f: Function) = 
            match f with 
            | _ when f.IsComposite -> sprintf "%s [shape=ellipse, fontsize=20, penwidth=2, peripheries=2];" f.Uid
            | _ when f.IsPrimitive -> sprintf "%s [shape=ellipse, fontsize=20, penwidth=2, size=0.6];" f.Uid
            | _ -> sprintf "%s [shape=ellipse, fontsize=20, penwidth=4];" f.Uid

        let varEdges (f: Function) (v: Variable) = 
            let inputIndex = f.Inputs |> Seq.map (fun v -> v.Uid) |> Set
            let outputIndex = f.Outputs |> Seq.map (fun v -> v.Uid) |> Set
            match inputIndex.Contains(v.Uid), outputIndex.Contains(v.Uid) with 
            | true, _ when v.IsParameter -> sprintf "%s -> %s [label=\"input param\"];" v.Uid f.Uid |> Some
            | _, true when v.IsParameter -> sprintf "%s -> %s [label=\"output param\"];" f.Uid v.Uid|> Some
            | true, _ -> sprintf "%s -> %s [label=input];" v.Uid f.Uid|> Some 
            | _, true -> 
                if f.Outputs.Count = 1 && f.Output.Uid = v.Uid then
                    sprintf "%s -> %s [label=output];" f.Uid v.Owner.Output.Uid |> Some
                else sprintf "%s -> %s [label=output];" f.Uid v.Uid|> Some            
            | _ -> None
        
        let varOwner (v: Variable) =
            match v.Owner with
            | null -> None
            | _ -> sprintf "%s -> %s [style=\"dotted\"];" v.Owner.Uid v.Uid |> Some

        let vars = Seq.append f.Inputs f.Outputs
        let funs = seq { 
                yield f
                yield f.RootFunction;
                yield! vars |> Seq.map (fun v -> v.Owner) |> Seq.filter (isNull>>not) 
            } 

        seq {        
            if f.Uid <> f.RootFunction.Uid 
            then yield sprintf "%s -> %s [label=\"root function\"];" f.RootFunction.Uid f.Uid
            yield! vars |> Seq.map varShape
            yield! vars |> Seq.map varLabel
            yield! vars |> Seq.map (varEdges f) |> Seq.choose id
            //yield! vars |> Seq.map varOwner |> Seq.choose id
            yield! funs |> Seq.map funLabel 
            yield! funs |> Seq.map funShape
        } |> Seq.distinct


# Using GraphViz inside an F# jupyter notebook

As far as I can tell there are restrictions on loading javascript by using the <code>Util.Html</code> command. I was not able to initialize [d3-graphviz]() either by including the cdn reference or by downloading the library files and referring to them locally. 

What did work was using [webpack]() to bundle the d3-grphviz library files together, along with some event handlers that will to serve as API endpoints; which is to say, instead of directly calling the <code>d3.graphviz</code> object we trigger an event with the dot notation as payload, which will then be handled from the bundled js.

This seems like a very roundabout way of doing things, and if a better way to integrate html/js with IfCntk comes up, such as a [widget creation schema](https://github.com/fsprojects/IfSharp/issues/126), I will be glad to revisit and hopefully provide a more elegant solution.

The webpack project can be found [here](). 

In [8]:
@"<script src='../../../d3-jupyter/dist/bundle.js'></script>" 
|> Util.Html |> Display

<div class="alert alert-info">While I include the <a>bundle.js</a> in the repository, you should remember that making changes to the <a>index.js</a> requires you to rebuild the project with webpack.</div>

Wrapper functions for accessing d3-graphviz through F#:

In [9]:
/// Provide jQuery paths to the html elements
/// you want the model graph and the node 
/// descriptions to appear
let initGraph infoPath graphPath = 
    sprintf "<script>$(document).trigger('INIT_D3', ['%s','%s']);</script>" infoPath graphPath
    |> Util.Html


/// Send dot notation to be rendered as a graph
let renderDot engine dotNotation jsonInfo = 
    sprintf "<script>$(document).trigger('RENDER_GRAPH', [`%s`,`%s`, '%s']);</script>" dotNotation jsonInfo engine
    |> Util.Html

/// Send a series of graphs to be 
/// sequentially animated into each other
let renderSeries engine (digraphs : string[]) = 
    digraphs
    |> Array.reduce(sprintf "%s','%s")
    |> fun graphs -> graphs,engine
    ||> sprintf "<script>$(document).trigger('RENDER_SERIES', [['%s'],'%s']);</script>" 
    |> Util.Html   

# Bringing the whole thing together

In [10]:
/// Extract notation for each CNTK.Function in model
/// combine in a string and send to graphviz for rendering
let displayGraphForModel hostId (model: Function) =
    let dotNotation =
        model
        |> decomposeFunction
        // |> Array.filter (fun f -> not f.IsComposite)
        |> Array.collect (extractGraphVizDotNotation>>Array.ofSeq)
        |> Array.distinct 
        |> Array.reduce(sprintf "%s\n%s")
        |> sprintf "digraph { %s }"
            
    initGraph "" ("#"+hostId) |> Display
    renderDot "dot" dotNotation "" |> Display   

In order to be able to show the d3-graphviz output we need a host for the svg inside the notebook. This is trivial to do in IfCntk, again by using the <code>Util.Html</code>:

In [32]:
let displayGraphHost hostId =
    """
<div id="hostId" 
     style="width: 100%; overflow-y: scroll; overflow-x: hidden; 
            border: solid lightblue 1px; border-radius: 15px; position: relative">    
</div>""".Replace("hostId", hostId)
    |> Util.Html |> Display

In [33]:
displayGraphHost "graph"
displayGraphForModel "graph" z

<div class="alert alert-info"> Feel free to exclude or include the composite function node in your graph, since it is seems generally redundant for so simple a model, where it's implied that all the component nodes are parts of a composite model.</div>

### Let's add some animation

d3-graphviz allows us to animate one graph into an other, simply by declaring a transition filter and making consecutive calls to the render function. Best results require consequent renderings be done after the current graph has finished rendering, so we cannot just call <code>renderGraph</code> a bunch of times and be done with it, which is why the RENDER_SERIES event is used. For javascript implementation details see [index.js]().

In [20]:
let displayGradualGraph hostId (model: Function) =

    initGraph "" ("#" + hostId) |> Display

    let dotNotation =
        model
        |> decomposeFunction 
        |> Array.filter (fun f -> not f.IsComposite)
        |> Array.map (extractGraphVizDotNotation>>Seq.reduce(sprintf "%s %s")) 
        |> Array.rev
        |> Array.scan (sprintf "%s %s") ""
        |> Array.map (sprintf "digraph { %s }")
           
    renderSeries "dot" dotNotation
    |> Display

In [34]:
displayGraphHost "gradualGraph"
displayGradualGraph "gradualGraph" z

<div class="alert alert-warning"><p>Animation quality can vary a bit, in the form of the animation going too fast inspite of the transition filter's delay and duration parameters.</p><p> I'm getting the best results by using a markdown cell as a host instead of the <code>HtmlOutput</code> result, maybe even call <code>$("svg").remove()</code> in between...</div>

# Adding an information panel next to the graph

d3 makes it pretty easy to hang events any element in your graph, so why not present a property dump for each node on mouseover, along with the graph? 

First we'll need a function to retrieve said property dump. Most CNTK objects implement an <code>.AsString()</code> function that returns a very short summary of the item, so we will use that when it's available. There also needs to be some special handling for list properties, and for the rest we can just call <code>.ToString()</code>.

In [42]:
open System.Collections.Generic

/// Active pattern to separate properties for special handling.
let (|IsEnumerable|IsDescribable|IsPrimitive|) (t: Type) = 
    if  t <> typeof<string> && 
        (typeof< IEnumerable<_> >).IsAssignableFrom(t) 
    then IsEnumerable
    else if t.GetMethods() 
            |> Array.exists (fun meth -> meth.Name = "AsString") 
    then IsDescribable
    else IsPrimitive

/// Helper function to convert any property to string,
/// always checking if .AsString() is available
let asString item =
    if isNull item then ""
    else
        match item.GetType() with
        | IsDescribable ->     
            item.GetType()
                .GetMethod("AsString", Array.empty)
                .Invoke(item, Array.empty).ToString()
        | _ -> item.ToString()

In [44]:
open System.Reflection

/// A simple info dump for CNTK nodes
/// <remarks> CNTK Helper function </remarks>
let describeNode (item: obj) =
    [|
        yield KeyValuePair("NodeType", item.GetType().Name)
        yield KeyValuePair("AsString", item |> asString)
        yield!
            item.GetType().GetProperties()
            |> Seq.map
                (fun prop ->
                    match prop.PropertyType with
                    | IsEnumerable -> 
                        prop.Name, 
                          (prop.GetValue(item) :?> IEnumerable<_>) 
                          |> function 
                          | list when list |> Seq.isEmpty -> "[]" 
                          | list -> 
                              list 
                              |> Seq.map (asString)                         
                              |> Seq.reduce (sprintf "%s, %s")
                    | IsDescribable -> prop.Name, prop.GetValue(item) |> asString
                    | IsPrimitive -> 
                        prop.Name,
                            try prop.GetValue(item) |> asString
                            with ex -> sprintf "%s" ex.Message)
            |> Seq.map (KeyValuePair)
    |]

<div class="alert alert-info">Mapping the result of <code>describeNode</code> to <code>KeyValuePair</code> is just there to make the eventual json serialization that we send to the javascript bundle a bit more readable.</div>

In order to display the node info table properly, we need expand our html host accordingly.

In [46]:
let displayGraphHostWithInfoPanel graphHostId infoPanelId =
    """
<div style="width: 100%; height: 700px; max-height: 700px; overflow-y: scroll; overflow-x: hidden; border: solid lightblue 1px; border-radius: 15px; position: relative">
    <div id="graphHostId" style="width: 100%; height: 100%;"></div>
    <div id="infoPanelId" style="width: 350px; max-width: 350px; position: absolute; top: 0; right: 0; background-color: white"></div>
</div>
    """.Replace("graphHostId",graphHostId).Replace("infoPanelId",infoPanelId)
    |> Util.Html |> Display

An we must of course include the code for producing the information dumps in our dot notation extraction function. In order to keep things from becoming to convoluted, we extract all the node descriptions and send them to the bundled javascript at once.

In [53]:
open Newtonsoft.Json

let displayGraphWithInfo graphHostId infoHostId (model: Function) =
    let funcs = model |> decomposeFunction //|> Array.filter (fun f -> not f.IsComposite)
    let vars = 
        funcs 
        |> Array.collect(fun f -> [|yield! f.Inputs; yield! f.Outputs|])
        |> Array.distinctBy (fun v -> v.Uid)
    
    let describe (tensor: obj) =
        tensor |> describeNode |> JsonConvert.SerializeObject
        
    let nodeInfo = 
        [|  yield "{"
            yield
                funcs 
                |> Array.map 
                    (fun f -> sprintf "\"%s\": %s" (f.Uid) (describe f)) //f |> describeNode |> JsonConvert.SerializeObject))                            
                |> Array.reduce(sprintf "%s,\n%s")
            yield ","
            yield
                vars 
                |> Array.map 
                    (fun v -> sprintf "\"%s\": %s" (v.Uid) (describe v)) // |> describeNode |> JsonConvert.SerializeObject))                            
                |> Array.reduce(sprintf "%s,\n%s")
            yield "}" |]
        |> Array.reduce(sprintf "%s\n%s")
        |> fun text -> text.Trim();
    
    let dotNotation =
        funcs
        |> Array.collect (extractGraphVizDotNotation>>Array.ofSeq)
        |> Array.distinct 
        |> Array.reduce(sprintf "%s\n%s")
        |> sprintf "digraph { %s }"
            
    initGraph ("#" + infoHostId) ("#" + graphHostId) |> Display
    renderDot "dot" dotNotation nodeInfo |> Display

In [54]:
displayGraphHostWithInfoPanel "graphWithPanel" "infoPanel"
displayGraphWithInfo "graphWithPanel" "infoPanel" z

In [37]:
module Engine =    
    let dot = "dot"
    let circo = "circo"
    let fdp = "fdp"
    let neato = "neato"
    let osage = "osage"
    let patchwork = "patchwork"
    let twopi = "twopi"

<div style="width: 100%; height: 700px; max-height: 700px; overflow-y: scroll; overflow-x: hidden; border: solid lightblue 1px; border-radius: 15px; position: relative">
    <div id="graph" style="width: 100%; height: 100%;"></div>
    <div id="info" style="width: 350px; max-width: 350px; position: absolute; top: 0; right: 0; background-color: white"></div>
    <div style="display: flex;position: absolute; top: 10px; left: 10px; align-items: center;">
        <span>Graphviz engine:</span>
        <select id="engineSelect" class="form-control" style="width: 150px">
            <option>dot</option>
            <option>circo</option>
            <option>fdp</option>
            <option>neato</option>
            <option>osage</option>
            <option>patchwork</option>
            <option>twopi</option>
        </select>
    <div>
</div>

In [39]:
let displayGradualGraph engine (model: Function) =
    
    let callJs engine graphs = 
        (graphs,engine)
        ||> sprintf "<script>$(document).trigger('RENDER_SERIES', [['%s'], '%s']);</script>"
        |> Util.Html  
        
    initGraph "" "#gradualGraph2" |> Display
    
    model
    |> decomposeFunction 
    //|> Array.filter (fun f -> not f.IsComposite)
    |> Array.map (extractGraphVizDotNotation>>Seq.reduce(sprintf "%s %s")) 
    |> Array.rev
    |> Array.scan (sprintf "%s %s") ""
    |> Array.map (sprintf "digraph { %s }")
    |> Array.reduce (sprintf "%s','%s")
    |> callJs engine    
    |> Display
            
displayGradualGraph Engine.dot z

In [40]:
let classifier = z

In [41]:
displayGradualGraph Engine.dot classifier

<div id="gradualGraph2" style="width: 100%; min-height: 700px; border: solid aliceblue 1px">
</div>

In [ ]:
type Tensor = 
| F of Function 
| V of Variable
with 
    member x.Uid = match x with F f -> f.Uid | V v -> v.Uid
    member x.AsString = match x with F f -> f.AsString() | V v -> v.AsString()

let decomposeTensor (root: Tensor) = 
        
        let visited = Dictionary<string, Tensor>()            

        let rec search (f: Tensor) = 
            visited.Add(f.Uid, f)
        
            seq {   
                match f with
                | F f' -> 
                    yield! f'.Inputs |> Seq.map V
                    yield! f'.Outputs |> Seq.map V
                    yield  f'.RootFunction |> F 
                | V v' -> 
                    yield f
                    if v'.Owner |> isNull |> not then 
                        yield v'.Owner |> F
            }
            |> Seq.filter (fun f -> visited.ContainsKey(f.Uid) |> not)
            |> Seq.iter search

        search root
    
        visited

In [ ]:
"""<script>$(document).on("NODE_CLICKED", 
    (e,d,i,n)=>{
        console.log(e,d,i,n);
});</script>""" |> Util.Html

In [ ]:
"../../../d3-jupyter/dist/bundle.js" |> System.IO.Path.GetFullPath